In [3]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML
import os
import sys
import pandas as pd
import numpy as np
import requests
import time
sys.path.append(os.path.dirname(os.getcwd()))
from api_calls import get_batch_ids, get_specific_data_of_sample, get_ids_in_batch, get_nomad_ids_of_entry, get_sample_description

url_base = "https://nomad-hzb-se.de"
url = f"{url_base}/nomad-oasis/api/v1"
token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN']

# Main buttons and outputs
load_button = widgets.Button(description="Load Data")
upload_and_process = widgets.Button(description="Upload and process Data (not reversable)", layout=widgets.Layout(width='800px', height='80px'))
unrecognized_files_widget = widgets.HTML(value="",layout=widgets.Layout(width='500px'))
out = widgets.Output()
out2 = widgets.Output()

# Create a container for the upload_and_process button
upload_button_container = widgets.Output()

# Batch selection widgets
batch_ids_list_tmp = list(get_batch_ids(url, token))
time.sleep(0.2)
batch_ids_list = []
for b in batch_ids_list_tmp:
    if "_".join(b.split("_")[:-1]) in batch_ids_list_tmp:
        continue
    batch_ids_list.append(b)

batch_ids = widgets.Select(options=batch_ids_list, description='Batches', layout=widgets.Layout(width='800px', height='80px'))
search_field = widgets.Text(description="Filter")
search = widgets.Button(description="Search")

# File upload widgets
uploader = widgets.FileUpload(
    accept='',
    multiple=True,
    description='Upload Files'
)

file_selector = widgets.SelectMultiple(
    options=[],
    description='Files:',
    disabled=False,
    layout=widgets.Layout(width='500px', height='200px')
)

# Global variables
upload_files = []
sample_id_buttons = []
output_areas = {}
sample_files_dict = {}
file_type_dict = {}  # Dictionary to store file type selections for each sample and file
selected_sample_id = None
raw_upload_data = None

def get_upload_folder(upload_id):
    for upload_folder in os.listdir(".."):
        if upload_id not in upload_folder:
            continue
        return upload_folder

def process_upload(upload_id):
    response = requests.post(f'{url}/uploads/{upload_id}/action/process', headers={'Authorization': f'Bearer {token}'})
    while True:
        time.sleep(2)
        response = requests.get(f'{url}/uploads/{upload_id}', headers={'Authorization': f'Bearer {token}'})
        if not response.json()["data"]["process_running"]:
            break
        with out2:
            print('processing')

def extract_filenames(upload_data):
    filenames = []
    if hasattr(upload_data, 'keys'):
        for key in upload_data.keys():
            if isinstance(upload_data[key], dict) and 'name' in upload_data[key]:
                filenames.append(upload_data[key]['name'])
            else:
                filenames.append(key)
    elif isinstance(upload_data, (list, tuple)):
        for item in upload_data:
            if hasattr(item, 'name'):
                filenames.append(item.name)
            elif isinstance(item, dict) and 'name' in item:
                filenames.append(item['name'])
            else:
                filenames.append(str(item))
    return filenames


def on_upload_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        try:
            global raw_upload_data
            raw_upload_data = change['new']
            filenames = extract_filenames(change['new'])

            # Prefilter files
            recognized_files = []
            unrecognized_files = []
            files_with_dots = []  # New list to track files with dots in the name

            for filename in filenames:
                # Check for dots in filename (excluding the extension)
                base_name = os.path.splitext(filename)[0]
                if '.' in base_name:
                    files_with_dots.append(filename)

                # Continue with your existing recognition logic
                filename_lower = filename.lower()
                if any(keyword in filename_lower for keyword in ['jv', 'eqe', 'mppt', 'sem', 'xrd']):
                    recognized_files.append(filename)
                else:
                    unrecognized_files.append(filename)

            # Update file selector with recognized files only
            file_selector.options = sorted(recognized_files + unrecognized_files)

            with out2:
                out2.clear_output()
                print(f"Uploaded {len(filenames)} files. {len(recognized_files)} recognized files added to selector.")

                # Create unrecognized files widget if needed
                if unrecognized_files:
                    unrecognized_html = "<div style='color: #d9534f; padding: 10px; border: 1px solid #d9534f; border-radius: 5px; margin-top: 10px;'>"
                    unrecognized_html += "<p><strong>The following files were not recognized:</strong></p>"
                    unrecognized_html += "<p>Make sure to select the correct type for them: jv, eqe, mppt, sem, xrd</p>"
                    unrecognized_html += "<ul style='max-height: 150px; overflow-y: auto;'>"
                    for file in unrecognized_files:
                        unrecognized_html += f"<li>{file}</li>"
                    unrecognized_html += "</ul></div>"

                    # Display the HTML using widgets.HTML
                    display(widgets.HTML(unrecognized_html))

                # Create files with dots warning widget if needed
                if files_with_dots:
                    dots_html = "<div style='color: #f0ad4e; padding: 10px; border: 1px solid #f0ad4e; border-radius: 5px; margin-top: 10px;'>"
                    dots_html += "<p><strong>Warning: The following filenames contain periods (.) which may cause issues:</strong></p>"
                    dots_html += "<p>It's recommended to use underscores (_) instead of periods in filenames.</p>"
                    dots_html += "<ul style='max-height: 150px; overflow-y: auto;'>"
                    for file in files_with_dots:
                        dots_html += f"<li>{file}</li>"
                    dots_html += "</ul></div>"

                    # Display the HTML using widgets.HTML
                    display(widgets.HTML(dots_html))

        except Exception as e:
            with out2:
                out2.clear_output()
                print(f"Error processing upload: {e}")
                

def on_selection_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected = change['new']
        with out2:
            out2.clear_output()
            if selected:
                print(f"Selected files: {', '.join(selected)}")
            else:
                print("No files selected")


def on_remove_button_click(sample_id, sample_select):
    def handle_click(b):
        global sample_files_dict

        # Get selected files to remove
        selected_files = list(sample_select.value)

        if selected_files:
            # Remove selected files from the sample's list
            sample_files_dict[sample_id] = [f for f in sample_files_dict[sample_id] if f not in selected_files]

            # Update the SelectMultiple widget
            sample_select.options = sample_files_dict[sample_id]

            # Add the removed files back to the file_selector
            file_selector.options = list(file_selector.options) + selected_files

            with out2:
                out2.clear_output()
                print(f"Removed files from {sample_id}:")
                print(f"Remaining files: {sample_files_dict[sample_id]}")

    return handle_click


def on_sample_button_click(sample_id, sample_select):
    def handle_click(b):
        global selected_sample_id, sample_files_dict
        selected_sample_id = sample_id
        selected_files = list(file_selector.value)

        if selected_files:
            # Update the dictionary by appending the selected files
            sample_files_dict[sample_id].extend(selected_files)

            # Update the SelectMultiple widget with the current files
            sample_select.options = sample_files_dict[sample_id]

            # Remove selected files from SelectMultiple widget
            file_selector.options = [f for f in file_selector.options if f not in selected_files]

            # Print the updated dictionary for verification
            with out2:
                out2.clear_output()
                print(f"Updated files for {sample_id}:")
                print(sample_files_dict[sample_id])
    return handle_click


def on_sample_button_first_click(sample_id, output_area):
    def handle_first_click(b):
        global sample_files_dict, file_type_dict

        # Create SelectMultiple widget for this sample
        sample_select = widgets.SelectMultiple(
            options=sample_files_dict[sample_id],
            description='',
            disabled=False,
            layout=widgets.Layout(width='300px', height='100px')
        )

        # Create Add button (for subsequent additions)
        add_button = widgets.Button(
            description="Add Files",
            layout=widgets.Layout(width='100px')
        )

        # Create Remove button
        remove_button = widgets.Button(
            description="Remove",
            layout=widgets.Layout(width='100px')
        )

        # Create container for file type dropdowns
        file_type_container = widgets.VBox([], layout=widgets.Layout(margin='0', padding='0'))

        # Define measurement types
        measurement_types = ['jv', 'eqe', 'mppt', 'sem', 'xrd', 'hy']

        # Function to update the file type dropdowns
        def update_file_types():
            # Initialize file_type_dict entries for this sample if they don't exist
            if sample_id not in file_type_dict:
                file_type_dict[sample_id] = {}

            # Create dropdown widgets for each file
            dropdown_widgets = []
            for file_name in sample_files_dict[sample_id]:
                # Determine default type based on filename
                default_type = 'hy'  # Default
                file_lower = file_name.lower()
                for mtype in ['jv', 'eqe', 'mppt', 'sem', 'xrd']:
                    if mtype in file_lower:
                        default_type = mtype
                        break

                # Initialize in dictionary if not present
                if file_name not in file_type_dict[sample_id]:
                    file_type_dict[sample_id][file_name] = default_type

                # Create dropdown for this file
                dropdown = widgets.Dropdown(
                    options=measurement_types,
                    value=file_type_dict[sample_id][file_name],
                    description='',
                    layout=widgets.Layout(width='80px', height='22px')
                )

                # Create observer to update dictionary when dropdown changes
                def make_observer(fname):
                    def observer(change):
                        file_type_dict[sample_id][fname] = change['new']
                    return observer

                dropdown.observe(make_observer(file_name), names='value')

                # Create row with filename and dropdown
                truncated_name = file_name[:20] + '...' if len(file_name) > 20 else file_name
                row = widgets.HBox([
                    widgets.HTML(f"<div style='width:200px; font-size:0.9em; overflow:hidden'>{truncated_name}</div>"),
                    dropdown
                ], layout=widgets.Layout(margin='0', padding='0', height='25px'))

                dropdown_widgets.append(row)

            # Update the container with all dropdown widgets
            file_type_container.children = tuple(dropdown_widgets)

        # Set up callbacks
        add_button.on_click(on_sample_button_click(sample_id, sample_select))
        remove_button.on_click(on_remove_button_click(sample_id, sample_select))

        # Observer for when files are added/removed
        def on_options_change(change):
            update_file_types()

        sample_select.observe(on_options_change, names='options')

        # Initial update of file types
        update_file_types()

        # Display the widgets in the output area
        with output_area:
            output_area.clear_output()

            # Create a horizontal layout with the buttons and sample_select on the left
            # and the file type container on the right
            display(widgets.HBox([
                # Left side: buttons and sample_select
                widgets.VBox([
                    widgets.HBox([
                        add_button,
                        sample_select,
                        remove_button
                    ])
                ]),

                # Right side: file type container with a header
                widgets.VBox([
                    widgets.HTML("<small style='margin-bottom:5px'><b>Recognized Type:</b></small>"),
                    file_type_container
                ], layout=widgets.Layout(margin='0 0 0 15px'))  # Add left margin for spacing
            ]))

        # Also handle the initial file transfer
        selected_files = list(file_selector.value)
        if selected_files:
            # Update the dictionary by appending the selected files
            sample_files_dict[sample_id].extend(selected_files)

            # Update the SelectMultiple widget
            sample_select.options = sample_files_dict[sample_id]

            # Remove selected files from file_selector
            file_selector.options = [f for f in file_selector.options if f not in selected_files]

            # Update file types after adding files
            update_file_types()

            with out2:
                out2.clear_output()
                print(f"Added files to {sample_id}:")
                print(sample_files_dict[sample_id])

    return handle_first_click


def on_upload_file(b):
    global sample_files_dict, raw_upload_data, file_type_dict
    upload_ids = []

    # Process files for each sample ID in the dictionary
    for sample_id, file_names in sample_files_dict.items():
        if not file_names:  # Skip if no files assigned to this sample
            continue

        # Get upload ID for this sample
        entry_id, upload_id = get_nomad_ids_of_entry(url, token, sample_id)
        time.sleep(0.2)
        upload_ids.append(upload_id)
        upload_folder = get_upload_folder(upload_id)

        # Process each file assigned to this sample ID
        for file_name in file_names:
            # Find the file in uploader.value by matching the name
            file_content = None

            # uploader.value is a tuple of dictionaries
            for file_data in uploader.value:
                if file_data['name'] == file_name:
                    file_content = file_data['content']
                    break

            if file_content is None:
                with out2:
                    print(f"Could not find content for file: {file_name}")
                continue

            # Split filename for processing
            file_name_parts = file_name.split(".")
            file_type = file_name_parts[-1]
            file_name_old = ".".join(file_name_parts[:-1])

            # Get measurement type from dropdown selection if available
            if sample_id in file_type_dict and file_name in file_type_dict[sample_id]:
                measurement_type = file_type_dict[sample_id][file_name]
            else:
                # Fallback to determining type based on file name
                measurement_type = "hy"  # Default
                file_lower = file_name.lower()
                if "jv" in file_lower:
                    measurement_type = "jv"
                elif "mppt" in file_lower:
                    measurement_type = "mppt"
                elif "eqe" in file_lower:
                    measurement_type = "eqe"
                elif "sem" in file_lower:
                    measurement_type = "sem"
                elif "xrd" in file_lower:
                    measurement_type = "xrd"

            # Create new file name with appropriate measurement type
            new_file_name = f"{sample_id}.{file_name_old}.{measurement_type}.{file_type}"

            with out2:
                print(f"Writing file: {new_file_name}")
                if not upload_folder:
                    display(widgets.HTML(f'<p style="color:red;">No write access to upload {upload_id}!!!</p>'))
                    display(widgets.HTML(f'<p style="color:red;">Check access and restart voila!!</p>'))
                    continue               
                else: 
                    print(upload_folder)

            # Write the file
            with open(f"../{upload_folder}/{new_file_name}", "wb") as f:
                f.write(file_content)

    # Process uploads
    for upload_id in set(upload_ids):
        process_upload(upload_id)

    with out2:
        print("Upload processed")
        print(f"Processed {len(upload_ids)} sample IDs with assigned files")
        
        
def on_load_button_clicked(b):
    global out, sample_id_buttons, output_areas, sample_files_dict
    out.clear_output()

    # Clear previous buttons and output areas
    sample_id_buttons = []
    output_areas = {}  # Change to dictionary to track by sample_id

    # Get sample IDs for the selected batch
    sample_ids = get_ids_in_batch(url, token, [batch_ids.value])
    sample_descriptions = list(get_sample_description(url, token, sample_ids).values())
    print(len(sample_descriptions),sample_descriptions)
    print(len(sample_ids),sample_ids)
    time.sleep(0.2)

    # Initialize the dictionary with empty arrays for each sample ID
    sample_files_dict = {sample_id: [] for sample_id in sample_ids}

    # Create a container for each sample ID with initially just the sample button
    for sample_id, description in zip(sample_ids, sample_descriptions):  
        sample_button = widgets.Button(  
            description=sample_id+" ["+description+"]",  
            layout=widgets.Layout(width='400px')  
        )

        # Create output area to hold the SelectMultiple and Remove button
        output_area = widgets.Output()
        output_areas[sample_id] = output_area

        # Set up callback for the sample button
        sample_button.on_click(on_sample_button_first_click(sample_id, output_area))

        # Add to list
        sample_id_buttons.append(widgets.VBox([
            sample_button,
            output_area
        ]))

    # Display the sample ID buttons
    with out:
        display(widgets.HTML("<h3>Select a Sample ID and Upload Files</h3>"))
        display(widgets.VBox([
            widgets.HBox([uploader]),
            #widgets.HTML("<p>Select files from the list below:</p>"),
            widgets.HBox([
                widgets.VBox([
                    # Add instruction text at the top of the file selector column
                    widgets.HTML("<div style='margin-bottom:5px; font-style:italic; color:#555;'>Select files from the list below, then click on a sample ID to assign them</div>"),
                    file_selector,
                    unrecognized_files_widget
                ],layout=widgets.Layout(min_width='600px')),
                widgets.VBox(sample_id_buttons)
            ])
        ]))

    # Show the upload_and_process button
    with upload_button_container:
        upload_button_container.clear_output()
        display(upload_and_process)

    # Print the initialized dictionary for verification
    # with out2:
    #    out2.clear_output()
    #    print("Initialized sample files dictionary:")
    #    print(sample_files_dict)
    
    
def on_search_clicked(b):  
    if not search_field.value:  
        batch_ids.options = batch_ids_list  
        return  

    search_term = search_field.value.lower()  # Convert search term to lowercase  
    batch_ids.options = [d for d in batch_ids_list if search_term in d.lower()]  #
    

# Register callbacks
load_button.on_click(on_load_button_clicked)
upload_and_process.on_click(on_upload_file)
search.on_click(on_search_clicked)
uploader.observe(on_upload_change, names='value')
file_selector.observe(on_selection_change, names='value')

# Display the main interface
display(widgets.VBox([
    search_field,
    search,
    batch_ids,
    load_button,
    out,
    upload_button_container,  # Use the container instead of the button directly
    out2
]))


# # Create a debug button  
# debug_button = widgets.Button(  
    # description="Debug: Show Files Dictionary",  
    # button_style='info',  
    # layout=widgets.Layout(width='300px')  
# )  
  
# # Debug output area  
# debug_output = widgets.Output()  
  
# # Debug button callback function  
# def on_debug_button_clicked(b):
    # with debug_output:
        # debug_output.clear_output()
        # print("Current sample_files_dict:")
        # for sample_id, files in sample_files_dict.items():
            # print(f"{sample_id}: {files}")

        # print("\nCurrent file_type_dict:")
        # for sample_id, file_types in file_type_dict.items():
            # print(f"{sample_id}:")
            # for file_name, file_type in file_types.items():
                # print(f"  - {file_name}: {file_type}")
  
# # Register the callback  
# debug_button.on_click(on_debug_button_clicked)  
  
# # Update the main display to include the debug button and output  
# # Add this to the end of your script to display the debug components  
# display(widgets.VBox([  
    # debug_button,  
    # debug_output  
# ]))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
